# Generation of the historical dataset for temperature estimation

In [ ]:
import numpy as np
from pandas import read_csv

In [ ]:
# Recovery of historical data from Klower et al. and Lee et al.

klower_data_df = read_csv("historical_data_from_klower.csv", delimiter=";")
klower_data = klower_data_df.values
klower_data_years = klower_data[:, 0]
klower_data_co2 = klower_data[:, 1]
klower_data_distance = klower_data[:, 2]
aeromaps_data_co2 = klower_data[:, 3]

print(klower_data_years)

In [ ]:
# Estimation of historical years

## Index for 2000 year
k2000 = 0
for k in range(0, len(klower_data_years)):
    if klower_data_years[k] < 2000:
        k2000 += 1

## Historical years
historical_years = np.zeros(k2000 + 1)
for k in range(0, k2000 + 1):
    historical_years[k] = klower_data_years[k]
print(historical_years)

In [ ]:
# Estimation of historical CO2 emissions

## Correction of CO2 emissions (commercial scope and life cycle emissions)
difference_coefficient = np.zeros(len(klower_data_years) - k2000)
for k in range(0, len(klower_data_years) - k2000):
    difference_coefficient[k] = aeromaps_data_co2[k + k2000] / klower_data_co2[k + k2000]
mean_difference_coefficient = np.mean(difference_coefficient)

## Historical CO2 emissions
historical_co2_emissions = np.zeros(k2000 + 1)
for k in range(0, k2000):
    historical_co2_emissions[k] = klower_data_co2[k] * mean_difference_coefficient
historical_co2_emissions[k2000] = aeromaps_data_co2[k2000]
print(historical_co2_emissions)

In [ ]:
# Estimation of historical energy consumption

co2_emission_factor = 88.7  # [gCO2/MJ] including life cycle
historical_energy_consumption = historical_co2_emissions / (
    co2_emission_factor * 10 ** (-12)
)  # [MJ]
print(historical_energy_consumption)

In [ ]:
# Estimation of historical non-CO2 emissions

lhv_kerosene = 44  # [MJ/kg]
emission_index_nox = 0.01514
emission_index_h2o = 1.231
emission_index_soot = 3e-5
emission_index_sulfur = 0.0012
historical_nox_emissions = (
    emission_index_nox * historical_energy_consumption / lhv_kerosene * 10 ** (-9)
)
historical_h2o_emissions = (
    emission_index_h2o * historical_energy_consumption / lhv_kerosene * 10 ** (-9)
)
historical_soot_emissions = (
    emission_index_soot * historical_energy_consumption / lhv_kerosene * 10 ** (-9)
)
historical_sulfur_emissions = (
    emission_index_sulfur * historical_energy_consumption / lhv_kerosene * 10 ** (-9)
)
print(historical_nox_emissions)

In [ ]:
# Estimation of distance before 1980

## Index for 1980 year
k1980 = 0
for k in range(0, len(klower_data_years)):
    if klower_data_years[k] < 1980:
        k1980 += 1

## Conversion from million kilometer to kilometer
klower_data_distance = klower_data_distance * 10**6

## Correction of distance
distance_coefficient = klower_data_distance[k1980] / klower_data_co2[k1980]
historical_distance = np.zeros(k2000 + 1)
for k in range(0, k1980):
    historical_distance[k] = klower_data_co2[k] * distance_coefficient
for k in range(k1980, k2000 + 1):
    historical_distance[k] = klower_data_distance[k]
print(historical_distance)

In [ ]:
# Historical dataset

historical_dataset = np.zeros((k2000 + 1, 7))
for k in range(0, k2000 + 1):
    historical_dataset[k, 0] = historical_years[k]
    historical_dataset[k, 1] = historical_co2_emissions[k]
    historical_dataset[k, 2] = historical_nox_emissions[k]
    historical_dataset[k, 3] = historical_h2o_emissions[k]
    historical_dataset[k, 4] = historical_soot_emissions[k]
    historical_dataset[k, 5] = historical_sulfur_emissions[k]
    historical_dataset[k, 6] = historical_distance[k]

np.savetxt("temperature_historical_dataset.csv", historical_dataset, delimiter=";")